In [9]:
from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np

In [10]:
def phi(a, b, t):
    if(a+b>=2*t):
        result = (a-t)*(b-t)
    else:
        result = -1/2*((a-t)**2+(b-t)**2)
    return result
def phi_(a, b, t):
    if(-a+b>=2*t):
        result = (-a-b)*(b-t)
    else:
        result = -1/2*((-a-t)**2+(b-t)**2)
    return result

Regularized problem:

In [11]:
size = 20
card = 5
sigma = np.genfromtxt("orl200-005-b.mat", skip_header=1, dtype='int')
sigma = sigma[0:size,0:size]
mu = pd.read_csv("orl200-005-b.txt",sep=" ", index_col=False, skiprows=1,names=['mu_i', 'ignore'])
rho = np.genfromtxt("orl200-005-b.rho")
rho = rho.item()
mu = mu.iloc[0:size,0]
bounds = pd.read_csv("orl200-005-b.bds", sep=" ", index_col=False,names=['l_i', 'u_i'])
bounds = bounds.transpose()
u_i = bounds.iloc[1,0:size]

In [12]:
t = 1
x_guess = np.zeros(size)
y_guess = np.ones(size)
cond = 1
while ((t >= 10**(-8)) & (cond>10**(-6))):
    m = Model('portfolio')
    vars_ = pd.Series(m.addVars(size))  #C_0, ..., C_size
    y = pd.Series(m.addVars(size))
    min1 = pd.Series(m.addVars(size))
    min2 = pd.Series(m.addVars(size))
    portfolio_risk = vars_.dot(sigma).dot(vars_)
    m.setObjective(portfolio_risk, GRB.MINIMIZE)
    m.addConstr(vars_.sum() <= 1, 'budget')
    #mu=np.asarray(mu)
    m.addConstrs(((0 <= y[i]) for i in range(size)), 'yG0')
    m.addConstrs(((y[i]<=1) for i in range(size)), 'yL1')
    m.addConstrs(((0<=vars_[i]) for i in range(size)), 'u_i')
    m.addConstrs(((vars_[i] <= u_i[i]) for i in range(size)), 'u_i')
    m.addConstr(size-card <= y.sum(), 'cardinality')
    m.addConstr(rho <= mu.dot(vars_), 'portf_return')
    vars_.start = x_guess
    y.start = y_guess
    #m.addGenConstrMin(min1, [vars_[i],y[i]], "minconstr")
    m.addConstrs((min1[i] == min_(vars_[i],y[i]) for i in range(size)), 'min1')
    m.addConstrs((min1[i]<=t for i in range(size)), 'proba1')
    #m.addConstrs((min2[i] == min_(-1*vars_[i],y[i]) for i in range(size)), 'min2')
    #m.addConstrs((min2[i]<=t for i in range(size)), 'proba2')
    m.update()
    m.optimize()
    x_guess= np.array([v.x for v in vars_])
    y_guess= np.array([v.x for v in y])
    t = t*0.01
    cond = max(abs(np.multiply(x_guess,y_guess)))


Optimize a model with 103 rows, 80 columns and 160 nonzeros
Model has 210 quadratic objective terms
Model has 20 general constraints
Variable types: 80 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-03, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+00, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 2e+01]
Presolve removed 101 rows and 60 columns
Presolve time: 0.00s
Presolved: 2 rows, 20 columns, 40 nonzeros
Presolved model has 210 quadratic objective terms
Variable types: 20 continuous, 0 integer (0 binary)
Presolve time: 0.06s
Presolved: 2 rows, 20 columns, 40 nonzeros
Presolved model has 210 quadratic objective terms
Root barrier log...

Ordering time: 0.00s

Barrier statistics:
 Free vars  : 19
 AA' NZ     : 2.100e+02
 Factor NZ  : 2.310e+02
 Factor Ops : 3.311e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Prim

Best objective 6.161606989172e+01, best bound 6.161606989172e+01, gap 0.0000%
Optimize a model with 103 rows, 80 columns and 160 nonzeros
Model has 210 quadratic objective terms
Model has 20 general constraints
Variable types: 80 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-03, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+00, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-08, 2e+01]
Presolve added 0 rows and 40 columns
Presolve removed 40 rows and 0 columns
Presolve time: 0.00s
Presolved: 63 rows, 120 columns, 180 nonzeros
Presolved model has 40 SOS constraint(s)
Presolved model has 210 quadratic objective terms
Variable types: 80 continuous, 40 integer (40 binary)

Root relaxation: objective 3.222243e+01, 170 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   32.22243    0   2

In [13]:
for v in vars_:
    if v.x > 0:
        print('\t%s\t: %g' % (v.varname, v.x))

	C3	: 0.0997102
	C4	: 0.124929
	C8	: 0.113257
	C16	: 0.110858
	C19	: 0.12089
